In [9]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from astropy.time import Time
import metpy.calc as mp
from metpy.units import units
from windrose import WindroseAxes
import matplotlib.cm as cm
from glob import glob 
from evFileProcessing import evFuncs

def pmelTransport(files):
    bigDs = []
    fig = plt.figure(figsize=(18,8))
    for file in files:
        ds = xr.open_dataset(file,decode_times=False)
        times = ds.time.values+ds.time2.values/1000/60/60/24
        t2 = [Time(time,format='jd').iso for time in times]
        ds['time3'] = pd.to_datetime(t2)
        ds = ds.where(ds.v_1206<1000)
        #plt.plot(ds.time3,np.cumsum(np.ravel(ds.sel(depth=20).v_1206)))
        #plt.ylim(-10000,100000)
        bigDs.append(ds)
    return bigDs

def readTracks(file):
    dfTracks = pd.read_csv(file)
    dfTracks['Date_S'] = dfTracks['Date_S'].astype(str)
    dfTracks['Datetime'] = pd.to_datetime(dfTracks['Date_S'] + ' ' + dfTracks['Time_S'])
    dfTracks = dfTracks.drop(['Date_S','Time_S'], axis=1)
    dfTracks = dfTracks.set_index('Datetime')
    return dfTracks

D1_4 = ['../data/physicalData/PMEL_2017_17ckp11a_wcp_vel.nc','../data/physicalData/PMEL_2018_18ckp11a_wcp_vel.nc']
D2_5 = ['../data/physicalData/PMEL_2017_17ckp1a_wcp_vel.nc','../data/physicalData/PMEL_2018_18ckp1a_wcp_vel.nc']
D6 = ['../data/physicalData/PMEL_2018_18ckp4a_wcp_vel.nc']

df1 = readTracks('../data/analysisFiles/TrackRegions1_linear2.csv')
#df1['linearHead'] = df1.linearHead+20
df2 = readTracks('../data/analysisFiles/TrackRegions2_linear2.csv')
df4 = readTracks('../data/analysisFiles/TrackRegions4_linear2.csv')
df5 = readTracks('../data/analysisFiles/TrackRegions5_linear2.csv')
df6 = readTracks('../data/analysisFiles/TrackRegions6_linear2.csv')

In [10]:
# Moorings
filesD1Y1 = glob('../data/2018/EVFiles/DAFT1/exports/Loaded*.csv')
filesD1Y2 = glob('../data/2019/EVFiles/DAFT1/exports/Loaded*.csv')
filesD2Y1 = glob('../data/2018/EVFiles/DAFT2/exports/Loaded*.csv')
filesD2Y2 = glob('../data/2019/EVFiles/DAFT2/exports/Loaded*.csv')
filesD3Y2 = glob('../data/2019/EVFiles/DAFT3/exports/Loaded*.csv')
dfD1 = evFuncs.evExports.readEvExports(filesD1Y1+filesD1Y2)
dfD1.name = 'D1'
dfD2 = evFuncs.evExports.readEvExports(filesD2Y1+filesD2Y2)
dfD2.name = 'D2'
dfD3 = evFuncs.evExports.readEvExports(filesD3Y2)
dfD3.name = 'D3'

In [22]:
dfD1

,layer,sA,datetime
datetime,,,
2017-08-08 04:00:15.000,1.0,32.652548,2017-08-08 04:00:15.000
2017-08-08 04:00:15.000,2.0,68.926749,2017-08-08 04:00:15.000
2017-08-08 04:00:15.000,3.0,611.180067,2017-08-08 04:00:15.000
2017-08-08 04:00:15.000,4.0,397.356299,2017-08-08 04:00:15.000
2017-08-08 04:00:15.000,5.0,617.044011,2017-08-08 04:00:15.000
...,...,...,...
2019-08-26 22:00:20.530,6.0,0.004899,2019-08-26 22:00:20.530
2019-08-26 22:00:20.530,7.0,0.002609,2019-08-26 22:00:20.530
2019-08-26 22:00:20.530,8.0,0.001068,2019-08-26 22:00:20.530


In [23]:
df1['SigBS_mean'] = 10**(df1.TS_mean/10)
df1a = df1.resample('2H').mean()[['linearHead','linearSpeed','SigBS_mean']]
dfD1a = dfD1.resample('2H').sum()
dfD1a['sa'] = dfD1a.sA/(4*np.pi*(1852**2))

In [28]:
dfComb = dfD1a.join(df1a)
dfComb['abund'] = dfComb.sa/dfComb.SigBS_mean
dfComb

,layer,sA,sa,linearHead,linearSpeed,SigBS_mean,abund
datetime,,,,,,,
2017-08-08 04:00:00,55.0,2012.391039,4.668964e-05,84.550000,0.186824,0.000017,2.814312
2017-08-08 06:00:00,55.0,3062.037683,7.104258e-05,35.272533,0.179200,0.000004,18.054505
2017-08-08 08:00:00,55.0,3121.626086,7.242509e-05,57.573794,0.220765,0.000041,1.778716
2017-08-08 10:00:00,55.0,3427.674720,7.952575e-05,67.183000,0.175364,0.000045,1.767651
2017-08-08 12:00:00,55.0,4347.558814,1.008681e-04,130.742083,0.161667,0.000005,20.200679
...,...,...,...,...,...,...,...
2019-08-26 14:00:00,54.0,641.576614,1.488527e-05,NaN,NaN,NaN,NaN
2019-08-26 16:00:00,54.0,1603.207560,3.719614e-05,NaN,NaN,NaN,NaN
2019-08-26 18:00:00,54.0,4591.314104,1.065234e-04,NaN,NaN,NaN,NaN
